In [1]:
import os
import time
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
isbns = pd.read_json(os.path.join('data','sledovat.json'), dtype={'M_isbn':str})['M_isbn'].drop_duplicates().to_list()
print(f"Položek ke stažení: {len(isbns)}")

Položek ke stažení: 2835


In [3]:
def scrape_dk(isbn):
    
    kniha = {
        'ISBN': isbn,
        'DK_date': datetime.datetime.now().replace(microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    }
    
    try:
        r = requests.get(f"""https://www.databazeknih.cz/search?q={isbn}""")
        soup = BeautifulSoup(r.text, "html.parser")
    except Exception as e:
        print(e)
        try:
            time.sleep(120)
            r = requests.get(f"""https://www.databazeknih.cz/search?q={isbn}""")
            soup = BeautifulSoup(r.text, "html.parser")
        except:
            return {}
        
    kniha['DK_titul'] = soup.find("title").text.split("-")[0].strip()
    if kniha['DK_titul'] == 'Vyhledávání | Databáze knih':
        kniha['DK_titul'] = None
        return kniha
    
    try:
        kniha['DK_rating'] = int(soup.find(class_=lambda c: c and c.startswith("hodnoceni")).text.split(' ')[0].strip())
    except:
        pass
    try:
        kniha['DK_ratings_count'] = int(soup.find(class_="ratingDetail").text.replace('hodnocení','').replace(' ',''))
    except:
        pass
    
    kniha['DK_tags'] = [s.text for s in soup.find_all(class_="tag")]
    
    try:
        tabulka = soup.find(class_="morePadding")
        for tr in tabulka.find_all('tr'):
            try:
                kniha[f"""DK_{tr.find_all('td')[0].text.strip().replace(" ","_")}"""] = int(tr.find_all('td')[1].text.replace("x","").replace(" ","").strip())
            except:
                pass
    except:
        pass
    
    return kniha

In [4]:
scrape_dk('978-80-257-0493-6')

{'ISBN': '978-80-257-0493-6',
 'DK_date': '2024-04-07 13:37:38',
 'DK_titul': 'Zatím dobrý',
 'DK_rating': 93,
 'DK_ratings_count': 558,
 'DK_tags': ['útěk',
  'Tři králové',
  'bratři Mašínové',
  'komunistický režim',
  'Magnesia Litera'],
 'DK_Právě_čtených': 13,
 'DK_Přečtených': 687,
 'DK_Čtenářské_výzvě': 71,
 'DK_Doporučených': 87,
 'DK_Knihotéce': 178,
 'DK_Chystám_se_číst': 422,
 'DK_Chci_si_koupit': 80,
 'DK_dalších_seznamech': 5}

In [5]:
current_date = datetime.datetime.now()
date_string = current_date.strftime("%Y_%m_%d")
print(date_string)

2024_04_07


In [6]:
if not os.path.exists(f'data_raw/databazeknih/{date_string}'):
    os.makedirs(f'data_raw/databazeknih/{date_string}')

In [7]:
dknih = []
count = 0
for i in isbns:
    count += 1
    dknih.append(scrape_dk(i))
    if count % 20 == 0:
        pd.DataFrame(dknih).to_json(os.path.join(f'data_raw/databazeknih/{date_string}',f'databazeknih_{date_string}_{(int(count/20)):04d}.json'))
        print(f'databazeknih_{date_string}_{(int(count/20)):04d}.json')
        dknih = []

databazeknih_2024_04_07_0001.json
databazeknih_2024_04_07_0002.json
databazeknih_2024_04_07_0003.json
databazeknih_2024_04_07_0004.json
databazeknih_2024_04_07_0005.json
databazeknih_2024_04_07_0006.json
databazeknih_2024_04_07_0007.json
databazeknih_2024_04_07_0008.json
databazeknih_2024_04_07_0009.json
databazeknih_2024_04_07_0010.json
databazeknih_2024_04_07_0011.json
databazeknih_2024_04_07_0012.json
databazeknih_2024_04_07_0013.json
databazeknih_2024_04_07_0014.json
databazeknih_2024_04_07_0015.json
databazeknih_2024_04_07_0016.json
databazeknih_2024_04_07_0017.json
databazeknih_2024_04_07_0018.json
databazeknih_2024_04_07_0019.json
databazeknih_2024_04_07_0020.json
databazeknih_2024_04_07_0021.json
databazeknih_2024_04_07_0022.json
databazeknih_2024_04_07_0023.json
databazeknih_2024_04_07_0024.json
databazeknih_2024_04_07_0025.json
databazeknih_2024_04_07_0026.json
databazeknih_2024_04_07_0027.json
databazeknih_2024_04_07_0028.json
databazeknih_2024_04_07_0029.json
databazeknih_2